In [1]:
# %cd ./filelists/CUB/
# !ln -s  ../../../CUB_200_2011/images ./images
# %cd ../../

!ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images

# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
# !ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/cars-dataset/images ./filelists/cars/images 
# !ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

In [3]:
!tmux ls

0: 1 windows (created Sun Jun 13 03:08:54 2021) [137x32] (attached)
1: 1 windows (created Sun Jun 13 04:00:24 2021) [80x23]


In [3]:
!python train.py --dataset cars --train_aug --method protonet --committed --stop_epoch 400

/bin/bash: python: command not found


In [3]:
!python wandb_restore.py --id vba5sgld --path ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/best_model.tar

wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210609_045359-vba5sgld
wandb: Resuming run aircrafts protonet
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/vba5sgld
wandb: Run `wandb off` to turn off syncing.

Checkpoint restored at  ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/best_model.tar
The model's epoch is  399
Please rename it to continue training

wandb: Waiting for W&B process to finish, PID 440
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20210609_045359-vba5sgld/logs/debug.log
wandb: Find internal logs f

In [4]:
# Do again with 200 dimensions
# Do again with 399.tar

!python save_features.py --dataset aircrafts --train_aug --method protonet --committed

Commit after saving features and testing.
checkpoint_dir: ckpts/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010
USE BN: True
outfile is features/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
0/40
10/40
20/40
30/40


In [5]:
!python test.py --dataset aircrafts --train_aug --method protonet --committed

Testing involves fine-tuning. Commit after testing.
novel_file features/aircrafts/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
600 Test Acc = 91.90% +- 0.35%


In [ ]:
!python train.py --dataset aircrafts --train_aug --method baseline --jigsaw --lbda 0.5 --committed

tracking in baseline train: True
USE pre-trained model: False
Checkpoint path: ckpts/aircrafts/_resnet18_baseline_aug_tracking_jigsaw_lbda0.50Adam_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210609_053152-2j3ciima
wandb: Syncing run honest-plant-71
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/2j3ciima
wandb: Run `wandb off` to turn off syncing.

About to start training. Last model and best model will be saved in wandb at every model save. 
 Run save_features.py and test.py after the training completes, with the same arguments
Epoch 1 | Batch 50/313 | Loss 3.908358 | Loss Cls 4.318188 | Loss Jigsaw 3.498529 | Acc Cls 0.017500 | Acc Jigsaw 0.

In [5]:
from methods.baselinetrain import BaselineTrain
from io_utils import model_dict
import torch.nn as nn

gpu = 0

model = BaselineTrain(model_dict["resnet18"], 10)
# if more than 1 gpu, syncbatchnorm

model = model.cuda(gpu)

model = nn.parallel.DistributedDataParallel(model, device_ids=[gpu], find_unused_parameters=True) 

tracking in baseline train: True
USE pre-trained model: False


RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

In [22]:
from methods.baselinetrain import BaselineTrain
from io_utils import model_dict
import torch.nn as nn
import torch
import os
import torch.distributed as dist
import torch.multiprocessing as mp

rank = 0
# dist.init_process_group(backend='nccl', init_method='env://', world_size=1, rank=rank)

def train(gpu=0):
    
    
    model = BaselineTrain(model_dict["resnet18"], 10)
    # if more than 1 gpu, syncbatchnorm here
    model = model.cuda(gpu)
    print(dir(model))
    model = nn.parallel.DistributedDataParallel(model, device_ids=[gpu], find_unused_parameters=True) 
    print(model.feature())

os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29534'
# world_size = 1 * 1
# mp.spawn(train, nprocs=1, args=())

train()

tracking in baseline train: True
USE pre-trained model: False


ModuleAttributeError: 'DistributedDataParallel' object has no attribute 'feature'

### TODO

0. BaselineTrain - test with parallel and without parallel. Test with 2 GPUs also.
1. Prototypical networks - check train.py and protonet.py, integrate DDP, integrate val_loss
2. MAML - same

### Testing the distributed loader, model, logging etc.

In [25]:
!python train.py --dataset CUB --train_aug --method baseline --committed --stop_epoch 400 --eval_interval 2 --parallel

params.parallel =  True
params.gpus =  1
params.devices =  0
One process runs on gpu:  0
Tracking in baseline train: True
Use pre-trained model: False
Checkpoint path: ckpts/CUB/_resnet18_baseline_aug_tracking_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-06-13 19:05:59.543682: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-06-13 19:05:59.546129: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.26
wandb: Syncing run morning-donkey-123
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/3qupe6wb
wandb: Run data is saved locally in /kaggle/working/fsl_s